In [1]:
#Import Necessary libraries
library('tidyverse')
library('dplyr')
source('functions.R')

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.3.6      v purrr   0.3.5 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.1      v stringr 1.4.1 
v readr   2.1.3      v forcats 0.5.2 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
train = readRDS("04a-wrangledTrain.rds")
train = train[-84211, ]
train_subset = feature_selection(train)

In [28]:
myTrain = function(data){
    model = lm(log(price)~.+age:fuel+age:drive+age:type+age:manufacturer, data = data)
    return(model)
}

In [29]:
myPredict= function(model, newdata){
    log_price_predict = predict(model, newdata=newdata, interval="prediction", level=0.5)
    return(exp(log_price_predict))
}

In [30]:
myScore= function(predict, data){
    intervalScore(predict, data$price, level = 0.5)
    }

In [31]:
myScore(myPredict(myTrain(train), train), train)

Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"


$summary
[1] 5.000000e-01 9.310044e+03 1.527171e+04 6.252339e-01

In [32]:
kFold(3, 123, train)

Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"


[[1]]
[[1]]$summary
[1] 5.000000e-01 9.336728e+03 1.526558e+04 6.259055e-01


[[2]]
[[2]]$summary
[1] 5.000000e-01 9.330884e+03 1.535117e+04 6.235985e-01


[[3]]
[[3]]$summary
[1]     0.500000  9276.167696 15281.546914     0.623214

In [33]:
kFold(3, 123, train_subset)

[[1]]
[[1]]$summary
[1] 5.000000e-01 1.057224e+04 1.765235e+04 6.013442e-01


[[2]]
[[2]]$summary
[1]     0.500000 10593.040858 17745.762947     0.597976


[[3]]
[[3]]$summary
[1] 5.000000e-01 1.051025e+04 1.770630e+04 5.967303e-01

In [34]:
myPredict= function(model, newdata){
    log_price_predict = predict(model, newdata=newdata, interval="prediction", level=0.8)
    return(exp(log_price_predict))
}

In [35]:
myScore= function(predict, data){
    intervalScore(predict, data$price, level = 0.8)
    }

In [36]:
kFold(3, 123, train)

Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"


[[1]]
[[1]]$summary
[1] 8.000000e-01 1.827185e+04 2.427488e+04 8.639516e-01


[[2]]
[[2]]$summary
[1] 8.000000e-01 1.826182e+04 2.432825e+04 8.621368e-01


[[3]]
[[3]]$summary
[1] 8.000000e-01 1.815192e+04 2.431728e+04 8.604297e-01

In [37]:
kFold(3, 123, train_subset)

[[1]]
[[1]]$summary
[1] 8.000000e-01 2.089820e+04 2.798951e+04 8.487412e-01


[[2]]
[[2]]$summary
[1] 8.000000e-01 2.094176e+04 2.800277e+04 8.495255e-01


[[3]]
[[3]]$summary
[1] 8.000000e-01 2.076946e+04 2.804635e+04 8.455576e-01

#### Holdout trial

In [38]:
holdo<-readRDS("04b-wrangledHoldout.rds")
holdo_subset<- feature_selection(holdo)

In [39]:
model_full <- myTrain(train)
model_subset<-myTrain(train_subset)

In [40]:
myPredict= function(model, newdata){
    log_price_predict = predict(model, newdata=newdata, interval="prediction", level=0.5)
    return(exp(log_price_predict))
}

In [41]:
myScore= function(predict, data){
    intervalScore(predict, data$price, level = 0.5)
    }

In [42]:
prediction_full<- myPredict(model_full, holdo)
predict_subset<- myPredict(model_subset, holdo_subset)

Warning message in predict.lm(model, newdata = newdata, interval = "prediction", :
"prediction from a rank-deficient fit may be misleading"


In [43]:
myScore(prediction_full, holdo)

$summary
[1] 5.000000e-01 9.284001e+03 1.511222e+04 6.239798e-01

In [44]:
myScore(predict_subset, holdo_subset)

$summary
[1] 5.000000e-01 1.052920e+04 1.748382e+04 6.003363e-01